# Daily Notebook #2

### Current State

- Current Course: Data Analyst in Python: Data Manipulation
- Todays Notes: learned various basics of data manipulation. I am already comfortable with most of it, but using the notebook today to practice.

### Goals

### Dataset for Today

y

### Code